<a href="https://colab.research.google.com/github/INA-95/Customer-Analysis/blob/main/online_fashion_platform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount

from google.colab import drive
drive.mount('/drive')
%cd /drive/MyDrive/Side_Project/side_project/Online_Fashion_Platform_BI_Project

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).
/drive/MyDrive/Side_Project/side_project/Online_Fashion_Platform_BI_Project


In [53]:
# library

import os
from glob import glob
import pandas as pd
from datetime import datetime

In [3]:
# import data
# glob
# https://tempdev.tistory.com/10
path = glob("*.csv")
df = pd.read_csv(path[0])
df.head()

,t_date,p_id,name,products,price,colour,brand,img,ratingCount,avg_rating,description,p_attributes,Return
0,2022-04-20,17048614,Khushal K Women Black Ethnic Motifs Printed Ku...,"Kurta, Palazzos, Dupatta",5099,Black,Khushal K,http://assets.myntassets.com/assets/images/170...,4522.0,4.418399,Black printed Kurta with Palazzos with dupatta...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32...",1
1,2023-09-03,16524740,InWeave Women Orange Solid Kurta with Palazzos...,"Kurta, Palazzos, Floral Print Dupatta",5899,Orange,InWeave,http://assets.myntassets.com/assets/images/165...,1081.0,4.119334,Orange solid Kurta with Palazzos with dupatta<...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32...",1
2,2023-01-07,16331376,Anubhutee Women Navy Blue Ethnic Motifs Embroi...,"Kurta, Trousers, Dupatta",4899,Navy Blue,Anubhutee,http://assets.myntassets.com/assets/images/163...,1752.0,4.161530,Navy blue embroidered Kurta with Trousers with...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ...",0
3,2021-03-19,14709966,Nayo Women Red Floral Printed Kurta With Trous...,"Kurta, Trouser, Dupatta",3699,Red,Nayo,http://assets.myntassets.com/assets/images/147...,4113.0,4.088986,Red printed kurta with trouser and dupatta<br>...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ...",0
4,2023-08-08,11056154,AHIKA Women Black & Green Printed Straight Kurta,Kurta,1350,Black,AHIKA,http://assets.myntassets.com/assets/images/110...,21274.0,3.978377,"Black and green printed straight kurta, has a ...","{'Body Shape ID': '424', 'Body or Garment Size...",1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14214 entries, 0 to 14213
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   t_date        14214 non-null  object 
 1   p_id          14214 non-null  int64  
 2   name          14214 non-null  object 
 3   products      14214 non-null  object 
 4   price         14214 non-null  int64  
 5   colour        14214 non-null  object 
 6   brand         14214 non-null  object 
 7   img           14214 non-null  object 
 8   ratingCount   6530 non-null   float64
 9   avg_rating    6530 non-null   float64
 10  description   14214 non-null  object 
 11  p_attributes  14214 non-null  object 
 12  Return        14214 non-null  int64  
dtypes: float64(2), int64(3), object(8)
memory usage: 1.4+ MB


In [35]:
# which product did they sell the most?
# sell the most : the number of selling product?

def most_selling(col:str):
    cnt_of_prd = df.groupby(col)[col].count().rename_axis('product_name').reset_index(name = 'count')
    most_selling_prd = cnt_of_prd.sort_values(by = 'count', ascending = False)
    return most_selling_prd

In [36]:
most_selling('products')

,product_name,count
285,Jeans,941
200,Dupatta,870
580,Saree,857
749,Top,805
844,Trousers,762
...,...,...
375,"Kurta, Palazzos, Floral Print Dupatta",1
376,"Kurta, Palazzos, Jacket",1
377,"Kurta, Pant, Dupatta",1
378,"Kurta, Pants, Dupatta",1


In [41]:
df1 = most_selling('products')

In [39]:
# which product did the most customer get a refund

def most_refund(col:str):
    return_prd = df.groupby(col)['Return'].sum().rename_axis('product_name').reset_index(name = 'total_return')
    most_return = return_prd.sort_values(by = 'total_return', ascending = False)
    return most_return

In [40]:
most_refund('products')

,product_name,total_return
285,Jeans,473
580,Saree,447
200,Dupatta,437
844,Trousers,388
749,Top,363
...,...,...
668,"Skirt, Midi Skirt, Pencil Skirt, Floral Printe...",0
505,"Nursing & Feeding, Pure Cotton, Top",0
507,Nursing Tops,0
205,Dupattas,0


In [42]:
df2 = most_refund('products')

In [44]:
# return rate

df3 = pd.merge(df1, df2, on = 'product_name', how = 'inner')
df3.head()

,product_name,count,total_return
0,Jeans,941,473
1,Dupatta,870,437
2,Saree,857,447
3,Top,805,363
4,Trousers,762,388


In [47]:
return_rate = (df3['total_return'] / df3['count']) * 100
df3['return_rate'] = return_rate
df3.head()

,product_name,count,total_return,return_rate
0,Jeans,941,473,50.265675
1,Dupatta,870,437,50.229885
2,Saree,857,447,52.158693
3,Top,805,363,45.093168
4,Trousers,762,388,50.918635


In [50]:
df3 = df3.sort_values(by = 'return_rate', ascending = False)
df3.head()

,product_name,count,total_return,return_rate
909,trousers,1,1,100.0
286,Chaniya Choli,2,2,100.0
403,"Yoga Set, Undershirt",1,1,100.0
596,"Cropped Top, Skirt",1,1,100.0
599,"Culotte, Jumpsuit",1,1,100.0


In [ ]:
# stedy seller
# steady : yearly? monthly? quarterly?
# customer constantly make a purchase : steady purchase rate? average purchase rate?

In [55]:
# convert t_date object into datetime

df['t_date'] = pd.to_datetime(df['t_date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14214 entries, 0 to 14213
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   t_date        14214 non-null  datetime64[ns]
 1   p_id          14214 non-null  int64         
 2   name          14214 non-null  object        
 3   products      14214 non-null  object        
 4   price         14214 non-null  int64         
 5   colour        14214 non-null  object        
 6   brand         14214 non-null  object        
 7   img           14214 non-null  object        
 8   ratingCount   6530 non-null   float64       
 9   avg_rating    6530 non-null   float64       
 10  description   14214 non-null  object        
 11  p_attributes  14214 non-null  object        
 12  Return        14214 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(7)
memory usage: 1.4+ MB


In [60]:
df['year'] = df['t_date'].dt.year
df['month'] = df['t_date'].dt.month
df['day'] = df['t_date'].dt.day

In [61]:
df[['t_date','year', 'month', 'day']]

,t_date,year,month,day
0,2022-04-20,2022,4,20
1,2023-09-03,2023,9,3
2,2023-01-07,2023,1,7
3,2021-03-19,2021,3,19
4,2023-08-08,2023,8,8
...,...,...,...,...
14209,2023-03-20,2023,3,20
14210,2021-07-16,2021,7,16
14211,2022-10-22,2022,10,22
14212,2022-10-29,2022,10,29


In [95]:
temp = df.groupby(['year', 'month', 'day','products'])['products'].count().to_frame()
temp.head()

products
year month day products                   
2021 1     1   Blouse                    1
               Dupatta                   2
               Jeans                     2
               Joggers, Jumpsuit         1
               Jumpsuit                  1

In [96]:
temp = temp.rename(columns = {'products':'num_purchase'})

In [97]:
temp.reset_index()

,year,month,day,products,num_purchase
0,2021,1,1,Blouse,1
1,2021,1,1,Dupatta,2
2,2021,1,1,Jeans,2
3,2021,1,1,"Joggers, Jumpsuit",1
4,2021,1,1,Jumpsuit,1
...,...,...,...,...,...
11751,2023,12,30,"Kurta, Trousers, Dupatta",1
11752,2023,12,30,Kurti,1
11753,2023,12,30,Shrug,1
11754,2023,12,30,"Top, Palazzos",1


In [98]:
# yearly products purchase trend

temp.groupby(['year', 'products'])['num_purchase'].sum()

year  products          
2021  ,                     2
      A-Line Maxi Skirts    1
      A-Line Mini Skirt     1
      A-Line Skirt          4
      A-Line Top            7
                           ..
2023  Yoga Set              1
      cardigan              2
      jumpsuit              1
      saree                 1
      shawl                 1
Name: num_purchase, Length: 1376, dtype: int64

In [99]:
# monthly products purchase trend

temp.groupby(['month', 'products'])['num_purchase'].sum()

month  products       
1      A-Line Skirt       2
       A-Line Skirts      1
       A-Line Top         2
       Anarkali Kurta     1
       Biker Shorts       2
                         ..
12     Tunic, Saree       1
       Tunic, Trousers    1
       Vest, Jacket       1
       Waistcoat          1
       Wrap Top           2
Name: num_purchase, Length: 2368, dtype: int64

In [100]:
# daily products purchase trend

temp.groupby(['day', 'products'])['num_purchase'].sum()

day  products                        
1    A-Line Maxi Skirts                   1
     A-Line Skirt                         1
     Anarkali Kurta                       5
     Bagh Printed Pure Silk Dupatta       1
     Biker Shorts                         1
                                         ..
30   Trouser                              1
     Trousers                            33
     Tunic                                4
     Windcheater, Crop Quilted Jacket     1
     sweatshirt                           1
Name: num_purchase, Length: 3395, dtype: int64